In [10]:
import importlib
import math
import os
import sys

import numpy as np
import pandas as pd
import torch
from datasets import Dataset

sys.path.append("../")

import src

importlib.reload(src)

from src.data_prep_utils import (  # noqa: E402
    conala_to_time_batches,
    load_time_sorted_conala,
    prep_for_hf,
)

importlib.reload(src.data_prep_utils)

from src.model import load_model_tok

importlib.reload(src.model)

from src.processing_utils import *

importlib.reload(src.processing_utils)


from src.training import train_evaluate

importlib.reload(src.training)

[nltk_data] Downloading package punkt to
[nltk_data]     /home/RDC/zinovyee.hub/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/RDC/zinovyee.hub/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


<module 'src.training' from '/usr/net/zinovyee.hub/IRTG/MLSC/MLSC_DD/notebooks/../src/training.py'>

In [47]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["WANDB_DISABLED"] = "true"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [1]:
MODEL = "CodeT5"
TRAIN_N = 900
BATCH_SIZE = 35
DECODER_LENGTH = 20
ENCODER_LENGTH = 15

TRAIN_ARGS = {
    "TRAIN_N" : TRAIN_N,
    "BATCH_SIZE" : BATCH_SIZE,
    "DECODER_LENGTH" : DECODER_LENGTH,
    "ENCODER_LENGTH" : ENCODER_LENGTH,
    "MODEL" : MODEL,
    "SEQ_TRAINER_ARGS" : {
        "output_dir": f"reports/{MODEL}_{TRAIN_N}_{BATCH_SIZE}/results",
        "num_train_epochs": 2,
        "do_train": True,
        "do_eval": True,
        "per_device_train_batch_size": 4,
        "per_device_eval_batch_size": 4,
        "learning_rate": 5e-4,
        "warmup_steps": 100,
        "weight_decay": 0.1,
        "label_smoothing_factor": 0.1,
        "predict_with_generate": True,
        "logging_dir": f"reports/{MODEL}_{TRAIN_N}_{BATCH_SIZE}/logs",
        "logging_steps": 100,
        "save_total_limit": 1,
        "save_strategy": "epoch",
        "logging_strategy": "epoch",
        "evaluation_strategy": "epoch",
        "load_best_model_at_end": False,
    },
}

In [2]:
df = load_time_sorted_conala("../data/raw/conala")
df = conala_to_time_batches(df, TRAIN_N, BATCH_SIZE)

train_dataset = prep_for_hf(df, -1)
test_dataset = prep_for_hf(df, 0)

NameError: name 'load_time_sorted_conala' is not defined

In [50]:
model, tokenizer = load_model_tok(MODEL, freeze=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(device)

cuda


In [51]:
train_data = prepare_hg_ds(
    train_dataset,
    tokenizer=tokenizer,
    max_input_length=ENCODER_LENGTH,
    max_output_length=DECODER_LENGTH,
)
validation_data = prepare_hg_ds(  # noqa: F405
    test_dataset,
    tokenizer=tokenizer,
    max_input_length=ENCODER_LENGTH,
    max_output_length=DECODER_LENGTH,
)

Map:   0%|          | 0/1427 [00:00<?, ? examples/s]

Map:   0%|          | 0/42 [00:00<?, ? examples/s]

In [52]:
train_evaluate(train_data, validation_data, TRAIN_ARGS, model, tokenizer)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


   eval_loss  eval_rouge1  eval_rouge2  eval_rougeL  eval_rougeLsum  \
0      5.064        0.181        0.028        0.154           0.154   

   eval_bleu  eval_gen_len  
0       0.02        10.571  


/home/RDC/zinovyee.hub/.local/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/RDC/zinovyee.hub/.conda/envs/encode_code/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3442, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_1829/2862972091.py", line 1, in <module>
    train_evaluate(train_data, validation_data, TRAIN_ARGS, model, tokenizer)
  File "/usr/net/zinovyee.hub/IRTG/MLSC/MLSC_DD/notebooks/../src/training.py", line 43, in train_evaluate
    trainer.train()
  File "/home/RDC/zinovyee.hub/.local/lib/python3.9/site-packages/transformers/trainer.py", line 1662, in train
    return inner_training_loop(
  File "/home/RDC/zinovyee.hub/.local/lib/python3.9/site-packages/transformers/trainer.py", line 1929, in _inner_training_loop
    tr_loss_step = self.training_step(model, inputs)
  File "/home/RDC/zinovyee.hub/.local/lib/python3.9/site-packages/transformers/trainer.py", line 2717, in training_step
    loss.backward()
  File "/home/RDC/zinovyee.hub/.conda/

In [64]:
summaries_after_tuning = generate_summary(test_samples, 
                                        model,
                                        tokenizer,
                                        ENCODER_LENGTH,
                                        DECODER_LENGTH)[1]
    
for i, description in enumerate(test_samples["output_sequence"]):
        print('_'*10)
        print(f'Original: {description}')
        
        print('\n')
        print(f'Summary before Tuning: {summaries_before_tuning[i]}')
        print('\n')
        print(f'Summary after Tuning: {summaries_after_tuning[i]}')
        print('\n')
        print('_'*10)
        print('\n'*2)

__________
Original: Removing duplicate characters from a string


Summary before Tuning: Join the set with foo.


Summary after Tuning: Python: How to convert a string to a string?


__________



__________
Original: finding index of an item closest to the value in a list that's not entirely sorted


Summary before Tuning: Returns the minimum element in an array.


Summary after Tuning: How to get the largest item in a list?


__________



__________
Original: (Django) how to get month name?


Summary before Tuning: Return today s string representation


Summary after Tuning: get current time


__________



__________
Original: sorting values of python dict using sorted builtin function


Summary before Tuning: Returns a sorted list of the keys in the dictionary.


Summary after Tuning: iterate over a dictionary in sorted order


__________



__________
Original: Convert an IP string to a number and vice versa


Summary before Tuning: Returns a string with the IPv6 address.


Summ

In [ ]:
loss

In [65]:
loss = []
rouge_1 = []
for i in range(1, 34): 
    print(i)
    test_dataset = prep_for_hf(full_df, i)
    validation_data_txt = test_dataset
    validation_data = validation_data_txt.map(
        lambda batch: batch_tokenize_preprocess(
            batch, 
            tokenizer=tokenizer,
            max_input_length=ENCODER_LENGTH,
            max_output_length=DECODER_LENGTH
        ),
        batched=True,
        remove_columns=validation_data_txt.column_names,
    )


    trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_data,
    eval_dataset=validation_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    )

    # ZERO - SHOT
    results_zero_shot = trainer.evaluate()
    results_zero_shot_df = pd.DataFrame(data=results_zero_shot, index=[0])[eval_columns_list]
    results_zero_shot_df.loc[0, :] = results_zero_shot_df.loc[0, :].apply(lambda x: round(x, 3))
    print(results_zero_shot_df)
    rouge_1.append(results_zero_shot_df['eval_rouge1'])
    loss.append(results_zero_shot_df['eval_loss'])


1


Map:   0%|          | 0/53 [00:00<?, ? examples/s]

/home/RDC/zinovyee.hub/.conda/envs/encode_code/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


   eval_loss  eval_rouge1  eval_rouge2  eval_rougeL  eval_rougeLsum  \
0      3.546        0.411        0.178        0.373           0.373   

   eval_bleu  eval_gen_len  
0      0.066        10.698  
2


Map:   0%|          | 0/40 [00:00<?, ? examples/s]

...

/home/RDC/zinovyee.hub/.conda/envs/encode_code/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


   eval_loss  eval_rouge1  eval_rouge2  eval_rougeL  eval_rougeLsum  \
0      4.162        0.277        0.074        0.247           0.248   

   eval_bleu  eval_gen_len  
0       0.03        12.071  
33


Map:   0%|          | 0/19 [00:00<?, ? examples/s]

/home/RDC/zinovyee.hub/.conda/envs/encode_code/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


   eval_loss  eval_rouge1  eval_rouge2  eval_rougeL  eval_rougeLsum  \
0       4.25        0.367        0.096         0.31           0.316   

   eval_bleu  eval_gen_len  
0      0.037        11.632  


In [ ]:
print(1)

In [ ]:
rouge_1